# DCF Valuation Walk-through (1997-2002)

This notebook demonstrates the process of creating a Discounted Cash Flow (DCF) valuation from a 5-year revenue forecast.

We will use the custom `ValuationDCF` and `Financials` classes, calling the methods sequentially to show how the valuation is built from the ground up.

## 1. Imports

First, we import our external libraries and, most importantly, our custom `Financials`, `ValuationDCF`, and `IOService` classes from their respective `.py` files.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from loguru import logger
from pathlib import Path

from valuation.analysis.financials import Financials
from valuation.analysis.valuation import ValuationDCF, ValuationAssumptions
from valuation.infra.file.io import IOService
from valuation.infra.loggers import configure_logging
from valuation.analysis.valuation import ANALYSIS_DIRECTORY

configure_logging()


## 2. Setup: Load Financial Assumptions and Base Financials (1997)

The valuation process requires a base year of financial data. We'll instantiate the `Financials` class, which loads the 1997 data by default and automatically calculates key ratios.

In [ ]:
assumptions = ValuationAssumptions()
print(assumptions)
base_financials = Financials()

print(f"Base Financials Loaded for Fiscal Year: {base_financials.fiscal_year}")
print(f"Base Revenue: ${base_financials.revenue:,.0f}")
print(f"Calculated Operating Margin: {base_financials.operating_margin:.2f}%")
print(f"Calculated CapEx as % of Sales: {base_financials.capex_to_sales:.2f}%")
print(80*"=")
print(base_financials.income_statement)
print(base_financials.balance_sheet)
print(base_financials.cashflow_statement)
print(80*"=")
print(base_financials.profitability_ratios)
print(80*"=")
print(base_financials.profitability_metrics)
print(80*"=")
print(base_financials.efficiency_ratios)




## 3. Setup: Load 5-Year Forecast (1998-2002)

Next, we load the 5-year revenue forecast generated by our `mlforecast` model.

In [ ]:
# --- Load the Forecast Data --- 
forecast_filepath = ANALYSIS_DIRECTORY / "forecasts_5year_1997_2002.csv"
forecast_col_name = 'LGBMRegressor/MinTrace_method-ols' # This is the reconciled column

try:
    # We use pandas directly, assuming the file is in the same directory as the notebook
    forecast_df = IOService.read(forecast_filepath)
    
    # Ensure 'ds' column is datetime
    forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    print(f"Successfully loaded forecast from {forecast_filepath}")
    print(f"Total rows: {len(forecast_df):,}")
    print(f"Date Range: {forecast_df['ds'].min().date()} to {forecast_df['ds'].max().date()}")
    print(f"Forecast column to be used: {forecast_col_name}")
    print(f"Unique series found: {forecast_df['unique_id'].nunique():,}")
    print("\n--- Forecast Head ---")
    print(forecast_df.head())
except Exception as e:
    print(f"Error loading forecast file: {e}")
    print("Please ensure 'forecasts_5year_1997_2002.csv' is in the same directory as this notebook.")
    forecast_df = None # Set to None to prevent downstream errors

## 4. Setup: Initialize `ValuationDCF` Class

With our `base_financials` and `forecast_df` ready, we can now initialize the `ValuationDCF` class. This class will hold all our assumptions and calculated data.

In [ ]:
# Set valuation parameters
valuation_date = pd.Timestamp('1997-01-01')

# Calculate Net Debt from financials
net_debt = base_financials.total_debt - base_financials.cash_and_equivalents

# Initialize the DCF class
if forecast_df is not None:
    dcf = ValuationDCF(
        financials=base_financials,
        assumptions=assumptions,
        forecast_df=forecast_df,
        forecast_col=forecast_col_name,
        valuation_date=valuation_date,
        wacc_override=None,          
        net_debt=net_debt
    )
else:
    print("\n🛑 DCF Class not initialized because forecast data failed to load.")
    dcf = None

## 5. The Valuation Process: Step-by-Step

Instead of calling `run_full_valuation()`, we will call each method individually to walk through the logic. The `loguru` logger will print detailed output at each stage.

### Step 1: Aggregate Weekly Revenue

In [ ]:
if dcf:
    dcf.aggregate_revenue()

### Step 2: Convert to Annual Revenue

Now that `dcf.company_revenue_weekly` is populated, the rest of the methods will work correctly. This step converts the weekly totals into the 5 annual figures needed for the DCF.

In [ ]:
if dcf:
    dcf.convert_to_annual()

### Step 3: Build DCF Model (Calculate FCF)

This is the core of the model. The class takes the annual revenue, applies the operating assumptions from `financials` (OpMargin, CapEx, NWC), and calculates the Free Cash Flow (FCF) for each of the 5 years. It then discounts these FCFs to their present value (PV).

In [ ]:
if dcf:
    dcf.build_dcf_model()

### Step 4: Calculate Terminal Value

After the 5-year forecast period, we calculate the terminal value using the perpetuity growth model. This value represents the sum of all future cash flows beyond Year 5, discounted back to Year 5, and then discounted back to the present.

In [ ]:
if dcf:
    dcf.calculate_terminal_value()

### Step 5: Calculate Enterprise Value

Enterprise Value (EV) is the sum of the Present Value of the 5-year forecast FCFs and the Present Value of the Terminal Value.

In [ ]:
if dcf:
    dcf.calculate_enterprise_value()

### Step 6: Bridge to Equity Value

Finally, we bridge from Enterprise Value to Equity Value by subtracting Net Debt and any other adjustments.

In [ ]:
if dcf:
    dcf.calculate_equity_value()

## 6. Post-Calculation: Analysis and Outputs

With the valuation complete, we can run the sensitivity analysis and generate our final reports.

### Step 7: Run Sensitivity Analysis

This method creates a 2-way data table showing how Enterprise Value changes based on different WACC and Terminal Growth Rate assumptions.

In [ ]:
if dcf:
    dcf.run_sensitivity_analysis()

### Step 8: Create Visualizations

This method generates the 9-panel dashboard and saves it as a `.png` file in our working directory.

In [ ]:
if dcf:
    dcf.create_visualizations()

### Step 9: Export Results

This method saves the key data tables (annual model, sensitivity, and summary) to `.csv` files.

In [ ]:
if dcf:
    dcf.export_results()

## Conclusion

The valuation is complete. We have successfully worked around the data-class incompatibility by manually aggregating the forecasts. We then used the `ValuationDCF` class to apply financial assumptions and generate a full DCF model with sensitivity analysis and visualizations. 

Check your directory for the output files:

- `dcf_valuation_dashboard.png`
- `dcf_model_annual.csv`
- `dcf_sensitivity_analysis.csv`
- `dcf_valuation_summary.csv`